In [48]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
plt.style.use("ggplot")
%matplotlib inline

In [49]:
train_df = pd.read_csv("data/train.csv").fillna("sterby")
test_df = pd.read_csv("data/test.csv").fillna("sterby")

In [50]:
train_df.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [51]:
X_train = train_df["comment_text"].values
y_train = train_df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test_df["comment_text"].values

In [52]:
yy = max(X_train, key=len)
list(X_train).index(yy)
max_len = 0
count = 0
maxi_len = 0
for jj in range(0,len(X_train)):
    #maxi_len = max(maxi_len,len(X_train[jj]))
    max_len += len(X_train[jj])
    count +=1
vv = max_len / count
arr = '\n'.join([''.join(str(row)) for row in X_train])   


y = Counter(arr.split())
len(y.keys())
ps = PorterStemmer()
 
sentence = arr
checks = {}
#words = word_tokenize(sentence)
words = y.keys()
for word in words:
    if word.isalpha():
    #word = re.sub('[^A-Za-z0-9]+', '', word)
        word = re.sub(r'\d+', '', word)
        if len(word) >= 3:
            try:
                checks[ps.stem(word)] = 0
            except: 
                print(len(word))
            
zz = len(checks)
maxi_len

4931


0

In [53]:
from keras.preprocessing import sequence
from keras.models import Model, Input
from keras.layers import Dense, Embedding, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.optimizers import *
from keras import regularizers


In [54]:
#max_features = 20000  # number of words we want to keep
max_features = int(zz)
#maxlen = 100  # max length of the comments in the model
#maxlen = int(vv)
maxlen = 100
batch_size = 128  # batch size for the model
embedding_dims = 32  # dimension of the hidden variable, i.e. the embedding dimension

In [55]:
tok = Tokenizer(num_words=max_features)
tok.fit_on_texts(list(X_train) + list(X_test))
x_train = tok.texts_to_sequences(X_train)
x_test = tok.texts_to_sequences(X_test)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Average train sequence length: {}'.format(np.mean(list(map(len, x_train)), dtype=int)))
print('Average test sequence length: {}'.format(np.mean(list(map(len, x_test)), dtype=int)))

159571 train sequences
153164 test sequences
Average train sequence length: 67
Average test sequence length: 60


In [56]:
x_train = sequence.pad_sequences(x_train, maxlen=int(maxlen))
x_test = sequence.pad_sequences(x_test, maxlen=int(maxlen))
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (159571, 100)
x_test shape: (153164, 100)


In [57]:
from keras.layers import *

comment_input = Input((int(maxlen),))

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
comment_emb = Embedding(int(max_features), embedding_dims, input_length=int(maxlen), 
                        embeddings_initializer="uniform", embeddings_regularizer= regularizers.l2(0.01))(comment_input)

# we add a GlobalMaxPooling1D, which will extract features from the embeddings
# of all words in the comment
#h = GlobalMaxPooling1D()(comment_emb)
#x = Conv1D(128, 5, activation='relu')(comment_emb)
#x = MaxPooling1D(5)(x)
#x = Conv1D(128, 5, activation='relu')(x)
#x = MaxPooling1D(5)(x)
#x = Flatten()(x)
#x = Dense(128, activation='relu')(x)
#x = Dense(64, activation='relu')(x)

x = Conv1D(256, 6, activation='relu')(comment_emb)
x = MaxPooling1D(6)(x)
x = Conv1D(256, 6, activation='relu')(x)
x = MaxPooling1D(6)(x)
x = Bidirectional(LSTM(100, return_sequences=True))(x)
#x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.3)(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.3)(x)
#x = Dense(128, activation='relu')(x)
#x = Dense(64, activation='relu')(x)
# We project onto a six-unit output layer, and squash it with a sigmoid:
output = Dense(6, activation='sigmoid')(x)

model = Model(inputs=comment_input, outputs=output)
#Nadam(0.003)
#Adagrad(0.005)
model.compile(loss='binary_crossentropy',
              optimizer=Nadam(0.003),
              metrics=['accuracy'])

In [58]:
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=19,shuffle=True, validation_split=0.20)


Train on 127656 samples, validate on 31915 samples
Epoch 1/19
127656/127656 [==============================] - 869s 7ms/step - loss: 0.3172 - acc: 0.9628 - val_loss: 0.1554 - val_acc: 0.9635
Epoch 2/19
100480/127656 [======================>.......] - ETA: 3:09 - loss: 0.1577 - acc: 0.9635

KeyboardInterrupt: 

In [ ]:
from keras.models import load_model
#model = load_model('final_18_March.h5')
prediction = model.predict(x_test)


In [ ]:
prediction2 = model.predict(x_train)

In [ ]:
model.evaluate(x_train,y_train)

In [ ]:
model.save('final2_march_19.h5')

In [ ]:
type(test_df['id'])

In [ ]:
type(prediction)

In [ ]:
pred = pd.DataFrame(prediction,columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"])
pred.head()

In [ ]:
final_test = test_df['id'].to_frame().join(pred)

In [ ]:
final_test.to_csv("submission.csv", index = False, sep=',', encoding='utf-8')